In [38]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import torch
import pyarrow.parquet as pq
from textDataset import TranslationDataset
from model import Transformer, Translator
from utils import load_tokenizer, load_data, build_tokenizer, get_config
from train import train
import numpy as np

In [42]:
config_path = 'config.json'
h = get_config(config_path)
ckpt_path = 'ckpt'

In [89]:
model = Transformer(
    N=h.N, 
    d_model=h.d_model, 
    num_heads=h.num_heads, 
    d_ff=h.d_ff, 
    vocab_size=h.vocab_size, 
    dropout=h.dropout, 
    max_len=h.max_len
)

In [56]:
data_path = '/home/dhhyun/data/test'
data = load_data(data_path)

1 .parquet files are found..


In [53]:
tokenizer = load_tokenizer(h.tokenizer_path)

loading tokenizer from file...
done


In [109]:
dataset = TranslationDataset(data, tokenizer, max_len=500)

In [115]:
batch = (
    torch.stack((dataset[0][0], dataset[1][0]), 0), 
    torch.stack((dataset[0][1], dataset[1][1]), 0), 
    torch.stack((dataset[0][2], dataset[1][2]), 0)
)
print(batch[0].shape)
print(batch[1].shape)
print(batch[2].shape)

torch.Size([2, 500])
torch.Size([2, 500])
torch.Size([2, 500])


In [116]:
pred = model(batch[0], batch[1])
ground_truth = batch[2]

In [132]:
ground_truth

tensor([[12828,  3705,    29,  3719,  4027,  5538,  6800,  3668,  5941,  3698,
            85,  6408,     3,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

In [131]:
ground_truth.reshape(1, -1)

tensor([[12828,  3705,    29,  3719,  4027,  5538,  6800,  3668,  5941,  3698,
            85,  6408,     3,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

In [112]:
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)


In [133]:
loss_fn(pred.reshape(-1, h.vocab_size), ground_truth.reshape(1, -1).squeeze(0))

tensor(9.6158, grad_fn=<AddBackward0>)